# 1. Load the data

In [1]:
import pandas as pd # pandas for data manipulation / analysis
import numpy as np # numpy for math
import matplotlib.pyplot as plt # pyplot for plotting and visualization
import datetime # datetime for computing times
import seaborn as sns # fancier plots

/Users/katherineli/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8 # set figsize for all future plots

In [3]:
# load data

data = pd.read_csv('daily_contract_production.csv', header = None)
data.columns = ['holding_company_id', 'project_company_id', 'contract_id', 'size_kwdc', 'created_on', \
                'updated_on', 'production_date', 'ato_date', 'actual_kwh', 'expected_kwh', \
                'weather_adjusted_expected_kwh']
extra = pd.read_csv('d_contracts.csv')

union = data.join(extra.set_index('id'), on = 'contract_id', rsuffix="DROP").filter(regex="^(?!.*DROP)")

union.head()

,holding_company_id,project_company_id,contract_id,size_kwdc,created_on,updated_on,production_date,ato_date,actual_kwh,expected_kwh,...,host_type,revenue_type,interconnection_type,registry_facility_name,cref_short_name,subscriber_orginization,system_yield_as_built,system_yield_finance,annual_degradation_engr,annual_degradation_finance
0,15,44,4FbAzTVD,25.13,49:53.6,53:14.9,00:00.0,11/29/18,NaN,42.908097,...,C&I,NEM,NEM,NON241648,NaN,NaN,1131.0,1131.0,0.005,NaN
1,14,50,56XBWDhT,40.00,49:53.6,53:14.9,00:00.0,9/28/18,NaN,71.340590,...,C&I,NEM,NEM,NON241695,NaN,NaN,954.7,1170.0,0.005,0.005
2,28,71,5Q6T8Cm7,6.70,49:53.6,53:14.9,00:00.0,11/28/18,0.139945,12.643548,...,Residential,NEM,NEM,NON241642,NaN,NaN,1250.0,1250.0,0.005,0.005
3,14,43,8EaZmYXK,26.18,49:53.6,53:14.9,00:00.0,3/10/18,23.113000,50.342822,...,C&I,CREF,CREF,NON241367,NCS07,Arcadia,1240.8,1327.0,0.005,NaN
4,15,44,8GniqiNd,77.52,49:53.6,53:14.9,00:00.0,7/12/18,NaN,142.074655,...,C&I,NEM,NEM,NON241708,NaN,NaN,1214.0,1214.0,0.005,NaN


In [4]:
# extract column info (types, values, etc.)

union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84216 entries, 0 to 84215
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   holding_company_id             84216 non-null  int64  
 1   project_company_id             84216 non-null  int64  
 2   contract_id                    84216 non-null  object 
 3   size_kwdc                      84216 non-null  float64
 4   created_on                     84216 non-null  object 
 5   updated_on                     84148 non-null  object 
 6   production_date                84216 non-null  object 
 7   ato_date                       84216 non-null  object 
 8   actual_kwh                     80982 non-null  float64
 9   expected_kwh                   84216 non-null  float64
 10  weather_adjusted_expected_kwh  84216 non-null  float64
 11  site_id                        84216 non-null  int64  
 12  in_service_date                41423 non-null 

In [5]:
# convert time-bearing columns to times (originally objects)

for i in range(4, 8):
    union[union.columns[i]] = pd.to_datetime(union[union.columns[i]])
    
union[union.columns[12]] = pd.to_datetime(union[union.columns[12]])
    
union.info()

ValueError: hour must be in 0..23

In [ ]:
union['subscriber_orginization'].value_counts()

# 2. Basic exploratory analysis

Examining the 'holding_company_id' column, we find that contracts, total system size, and amount of data are all pretty unevenly distributed between holding companies. This finding, itself, is not surprising, but the discrepancy between the three measures may be interesting to note. 

For example, holding company 3 may not have many contracts, but they have a large mass of solar system which produces a lot of data. 

Holding companies 5, 8, and 14 have a lot of data from many smaller contracts. 

Holding companies 24, 30, 32, 36 have many small contracts, but not a lot of data points.

From the 'project_company_id' column, company 6 dominates in size, few large contracts with a lot of data. 

Projects 10 and 37 have a number of large to medium sized contracts with a lot of data.

Projects 43, 44, and 56 have a lot of data with less size and less contracts.

Projects 41, 48, 54, 77, and 85 have many small contracts with not very much data.

In [ ]:
# plot how much data we have from each contract

contracts = union['contract_id'].value_counts() # count data points per contract and store it
contracts.plot(kind = 'bar', figsize = (15, 8))

Examining a simple count of contract IDs, we find many modes - 899 makes sense because data started being collected only on Jan 1, 2019 (899 days before this dataset was extracted), but others may be abnormal.

In [ ]:
contracts.value_counts()[contracts.value_counts() > 1]

In [ ]:
for elem in contracts[contracts == 383].index:
    print(union[union['contract_id'] == elem][['production_date', 'ato_date']])

As we can see, there are also a number of contracts which only started collecting data on 6/1/2021, 1/1/2021, 1/1/2021, 3/1/2020, 4/1/2020, 4/1/2021, 10/1/2019, 6/1/2020, etc. (in order of frequency) regardless of their ato dates (which often don't match the first production date). 

In [ ]:
# find any discrepancies between recorded ages of each contract (from ato date to today) and no. data points
# (should be one data point per day, but most don't match)

group = union.groupby('contract_id') # group data by contract
ato_unique = group.apply(lambda x: x['ato_date'].unique()[0]) # get a series of ato dates for each contract
system_age = pd.to_datetime(datetime.date.today()) - ato_unique # find age by subtracting from today
pseudo_age = pd.Series() # psuedo age bc some older contracts only started collecting data on Jan 1 2019
for contract in contracts.index: # set any contracts w/ ato date before Jan 1 2019 to 899 days old
    if ato_unique[contract] < pd.to_datetime(datetime.date(2019, 1, 1)):
        pseudo_age[contract] = pd.to_datetime(datetime.date(2021, 6, 18)) - pd.to_datetime(datetime.date(2019, 1, 1))
    else:
        pseudo_age[contract] = system_age[contract]
        
differences = {} # create a dictionary to store differences between age and datapoints per contract (above)

print("Contract \t Age \t Count \t Difference")
for i in range(len(pseudo_age)):
    print(f"{pseudo_age.index[i]} \t {pseudo_age[i].days} \t {contracts[pseudo_age.index[i]]} \t \
{contracts[pseudo_age.index[i]] - pseudo_age[i].days}")
    differences[pseudo_age.index[i]] = contracts[pseudo_age.index[i]] - pseudo_age[i].days

In [ ]:
sum(d == 0 for d in differences.values())

After counting the number of data points per contract, we want to compare to the number we expect based on the sage of each solar system. We calculate this by subtracting the ato date from the date this data was delivered. As we can see above, there are many discrepancies - only 34 out of 200 contracts have the expected number of datapoints! However, most of these values are negative, meaning we have missing data (as opposed to extra/likely duplicated data).

In [ ]:
sum(sorted(differences.values())[:-39])

We seem to be missing around 1/10 (8223/83019) of the data we expect to have. Next, we will look into extreme values (negative, too, but mostly positive), find and eliminate duplicate data.

# how to deal with this?

In [ ]:
# find largest discrepancies, positive and negative, for further examination

print(sorted(differences.items(), key = lambda item: item[1])[:10], "\n", \
sorted(differences.items(), key = lambda item: item[1])[-10:])

In [ ]:
# use this cell to filter dataframe for problematic contracts and examine them individually

union[union['contract_id'] == 'd2Ww9Bvv']

After examining some of the most extreme values, we find that, indeed, the contracts with very positive discrepancies seem to have some repeated production dates. Those with negative values are missing data. Many of the production dates start around a month or so after the ato date. Next, rows with duplicated production dates should be removed. With duplicated production dates, they seem to be mismatched on created_on and actual_kwh, with earlier (presumably non-updated) created_on dates corresponding to higher actual_kwh readings. These earlier readings will be deleted and only more recent created_on data will be maintained.

In [ ]:
# delete duplicated data
duplicated = ['RZdqcB2k', 'Giw4Zk3G', 't6Pwm8dh', 'hFjQMQMM', 'd2Ww9Bvv']

for contract in duplicated:
    duplicates = union[union['contract_id'] == contract]['production_date']
    union.drop(duplicates[duplicates.duplicated(keep = 'last')].index, inplace = True)
    
group = union.groupby('contract_id') # re-group data by contract since some rows were deleted

Next, we will look into the breakdown of contract type for solar system (commercial vs residential).

In [ ]:
# count unique contracts and type breakdowns for original data and supplementary data (slightly different?)

id_unique = group.apply(lambda x: x['host_type'].unique()[0])
print(id_unique.value_counts())
print(extra['host_type'].value_counts())

Only one data point is residential, so we will consider it an anomaly and remove it.

In [ ]:
union.drop(union[union['host_type'] == 'Residential'].index, inplace = True)
group = union.groupby('contract_id') # re-group again by contract since some rows were deleted

In [ ]:
# get an idea of solar system sizes

size_unique = group.apply(lambda x: x['size_kwdc'].unique()[0])
plt.hist(size_unique)

Next, observe contract sizes: the histogram below is heavily skewed left w/ many small contracts and a few big ones.

In [ ]:
# plot how many systems have been opened each month (total) - to get an idea of monthly trends

month_unique = group.apply(lambda x: x['ato_date'].dt.month.unique()[0])
plt.hist(month_unique, bins = 12)

We also plot a histogram of ato dates to see if there is a trend in the month a solar system tends to be created. As we can see, most are created in December as companies rush to meet year-end quotas, and not many are opened in January following. The peak in May is likely due to an influx of new contracts this year, May 2021, with our data for 2021 only leading up to mid-June.

In [ ]:
# plot how many systems were opened each month since Sep 2016

date_unique = pd.DataFrame() # new dataframe to store data by ato year and month
date_unique['year'] = group.apply(lambda x: x['ato_date'].dt.year.unique()[0]) # columns for year
date_unique['month'] = group.apply(lambda x: x['ato_date'].dt.month.unique()[0]) # and month
date_unique['count'] = 1 # count to keep track
agegroups = date_unique.groupby([date_unique['year'], date_unique['month']]).count()['count'] # count occurrences

# for months with 0 occurences, still add them as indices with values/counts of 0 for the bar plot
year = 2016
month = 9
while year < 2021:
    try:
        agegroups[(year, month)]
        if month == 12: 
            year += 1
            month = 1
        else: month += 1
    except:
        agegroups[(year, month)] = 0
        if month == 12: 
            year += 1
            month = 1
        else: month += 1
agegroups.sort_index(inplace = True)

agegroups.plot(kind = 'bar', figsize = (15, 8))

Breaking down our ato plot by year as well, we can see that more systems tend to be created each year as the company grows. Systems tend to peak in December, as observed before, and drop significantly for January. The fact that we only have data from September of 2016 may have also inflated the number in the month-only histogram above, causing the peak there. Interestingly, there are some months where no systems are created. Others, very few. However, evident by the peaks, the company must have the capacity to produce new systems quite quickly, making the lows potentially concerning in terms of demand. The peak this May may be due to the big deal with Franklin Park Infrastructure closed earlier this year. Below shows a cumulative plot of total systems over time (as opposed to only new systems created).

In [ ]:
# cumulative number of systems since Sep 2016

agegroups.cumsum().plot(kind = 'bar', figsize = (15, 8))

Next, we will begin looking into production, a key variable of interest.

In [ ]:
# get total production of each solar system

siteproduction = {contract: 0 for contract in set(union['contract_id'])} # create a dict to store values

# populate by summing production values for each contract id
for i in union.index:
    if np.isnan(union['actual_kwh'][i]): # skip nans
        continue
    siteproduction[union['contract_id'][i]] += union['actual_kwh'][i]

In [ ]:
# avg production of each system (calculated by total production of a system divided by no. data points)
# may be inaccurate as no. data points don't always match no. production days (missing data)

avgsiteproduction = {}
# convert each contract's total production to avg by dividing by a count of datapoints per contract
for elem in siteproduction.keys():
    avgsiteproduction[elem] = siteproduction[elem] / union['contract_id'].value_counts()[elem]

In [ ]:
# plot histogram of average solar system production (skewed left w/ many smaller values and some large ones)

plt.hist(avgsiteproduction.values())

This plot of average system production looks very similar to the histogram of sizes from above - this makes sense. The plot is again heavily skewed left with a few systems producing very highly and most producing smaller amounts on average. Below, plotting size against production directly, we do see that the bigger systems have higher max production values compared to smaller ones. However, the variance is also higher. The trend is that production does increase with size, as we would expect. Some smaller contracts also occasionally produce really high power.

In [ ]:
sns.regplot(union['size_kwdc'], union['actual_kwh'])

In [ ]:
# plot each contract's size versus average (daily) production for comparison

size_production = pd.DataFrame() # combine to a new dataframe to plot using pandas
size_production['production'] = pd.Series(avgsiteproduction) # cast production dict as series and add column
size_production['size'] = size_unique # size is already a series from earlier, so just add it
size_production.sort_values(by = 'size').plot(kind = 'bar', figsize = (15, 8))

Plotted individually by contract (above), again, size corresponds with average production. Below, the ratios of production/size per system are also pretty randomly distributed, if not quite uniform. However, there is one contract with net negative production.

In [ ]:
# find and plot ratios of production per size

size_production['ratio'] = pd.Series(avgsiteproduction)/size_unique # new column to dataframe for ratio
size_production['ratio'].plot(kind = 'bar', figsize = (15, 8))

In [ ]:
print(min(size_production['ratio'].items(), key = lambda item: item[1])) # what's the one super negative value?
print(size_production['ratio'].mean()) # and average multiplier

In [ ]:
# examine the anomalous contract w/ net negative production

jZWEeuhJ = union[union['contract_id'] == 'jZWEeuhJ']
jZWEeuhJ[jZWEeuhJ['actual_kwh'] < 0] # filter for only negative production values - some EXTREMELY negative

Looking further into the contract with negative production, we see, in fact, some VERY extreme negative values. We should probably check if there are more extreme values in the dataset and, if so, delete them so they don't skew our data.

In [ ]:
# find all datapoints with actual production values less than -10 (concerning - for further inspection)

anomalies = {}

for i in union.index:
    if data['actual_kwh'][i] < -10:
        try:
            anomalies[data['contract_id'][i]].append((data['production_date'][i], data['actual_kwh'][i]))
        except:
            anomalies[data['contract_id'][i]] = [(data['production_date'][i], data['actual_kwh'][i])]
            
for elem in anomalies.keys():
    print(elem)
    for val in anomalies[elem]:
        print(val)

In [ ]:
union.drop(union[union['actual_kwh'] < -10].index, inplace = True)
union.drop(union[union['actual_kwh'] > 100000].index, inplace = True)

group = union.groupby('contract_id') # re-group for deletion

The plots below also show the clear relationship between size and production and lack thereof between either and ratio.

In [ ]:
np.mean(size_production['ratio'])

In [ ]:
1423/(365)

In [ ]:
size_production['production'] = size_production['production']/900
size_production['size'] = size_production['size']/300
size_production['ratio'] = pd.Series(avgsiteproduction)/size_unique 

size_production.plot(kind = 'bar', figsize = (15, 8))

In [ ]:
size_production.sort_values(by = 'ratio').plot(kind = 'bar', figsize = (15, 8))

In [ ]:
size_production.sort_values(by = 'size').plot(kind = 'bar', figsize = (15, 8))

In [ ]:
size_production.sort_values(by = 'production').plot(kind = 'bar', figsize = (15, 8))

We also want to check discrepancies between creation and last update to ensure our data is valid and look for potential non-comms. 

# wasn't the metric for 'problematic' 2 days??

In [ ]:
# next, find datapoints with a lag between creation and last update (may be a sign of non-comms)

discrepancies = {}
for i in union.index:
    discrepancies[i] = union['updated_on'][i] - union['created_on'][i]

In [ ]:
# create a new column in our data for discrepancies - flag any longer than 7 days (most of them???)

union['discrepancy'] = discrepancies.values()
union[union['discrepancy'] > pd.Timedelta('7 days')][['created_on', 'updated_on', 'discrepancy']]

In [ ]:
union['discrepancy'].dt.days.value_counts()

We also want to plot production as a function of time. The first plot below shows total production of all systems over time (including / not factoring out the growing number of systems), the second shows average production each month since the beginning of 2019, when we started collecting data, and the final plot shows average monthly production irrespective of year. As we would expect, these graphs all fluctuate with seasons, peaking in summer with more sunlight exposure and dropping in winter with less.

In [ ]:
# plot total solar energy production as a function of time

production_date = pd.DataFrame()
production_date['year'] = group.apply(lambda x: x['production_date'].dt.year)
production_date['month'] = group.apply(lambda x: x['production_date'].dt.month)
production_date['production'] = group.apply(lambda x: x['actual_kwh'])
productiongroups = production_date.groupby([production_date['year'], production_date['month']])

productiongroups.sum()['production'].plot(kind = 'bar', figsize = (15, 8))

In [ ]:
# average energy production per contract as a function of time - cyclic with seasons

productiongroups.mean()['production'].plot(kind = 'bar', figsize = (15, 8))

In [ ]:
# average production per month disregarding year

productionmonth = production_date.groupby(production_date['month'])
productionmonth.mean()['production'].plot(kind = 'bar', figsize = (15, 8))

Next, we'll also plot production by ato dates. In the initial graph, it does seem like production by systems created this year is slightly higher than those from other years, but this could be due to chance, and it's generally pretty uniform over time. In the next graph by month only, there seems to be slightly lower production during later months compared to earlier ones, but again, it doesn't look like a big difference, and the graph is generally pretty uniform, meaning no real difference in production quality between months or years.

In [ ]:
# plot average production of systems opened in each month since Sep 2016

ato_date = pd.DataFrame()
ato_date['year'] = group.apply(lambda x: x['ato_date'].dt.year)
ato_date['month'] = group.apply(lambda x: x['ato_date'].dt.month)
ato_date['production'] = group.apply(lambda x: x['actual_kwh'])
ato_date['ratio'] = group.apply(lambda x: x['actual_kwh']/x['size_kwdc'])
atogroups = ato_date.groupby([ato_date['year'], ato_date['month']])
atoplot = atogroups.mean()['ratio']

year = 2016
month = 9
while year < 2021:
    try:
        atoplot[(year, month)]
        if month == 12: 
            year += 1
            month = 1
        else: month += 1
    except:
        atoplot[(year, month)] = 0
        if month == 12: 
            year += 1
            month = 1
        else: month += 1
atoplot.sort_index(inplace = True)

atoplot.plot(kind = 'bar', figsize = (15, 8))

In [ ]:
# average production of systems opened each month since Sep 2016 (by ato date) disregarding year

atomonth = ato_date.groupby(ato_date['month'])
atomonth.mean()['ratio'].plot(kind = 'bar', figsize = (15, 8))

We also might be interested in production over time. It's not very practical to generate a new graph for every contract (though it may be useful to do it for specific randomly chosen / bigger contracts), so we'll just assign each contract an age and plot age against production to track potential degradation.

In [ ]:
# add a column to dataframe for age

union['system age'] = union.apply(lambda x: system_age[x['contract_id']].days, axis=1)

In [ ]:
# scatterplot of age vs production

#sns.regplot(union['age'], union['actual_kwh'])

from scipy import stats

degradation = union[['system age', 'actual_kwh']].dropna()

# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(degradation['system age'], degradation['actual_kwh'])

# use line_kws to set line label for legend
ax = sns.regplot(x = 'system age', y = 'actual_kwh', data = degradation, color='b', \
                 line_kws={'label':"y={0:.1f}x+{1:.1f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()

In [ ]:
print(slope) # very naive calculation of degradation in units of kwh/day

In [ ]:
dpy = slope*365 # degradation per year
dpy

In [ ]:
ppy = np.mean(union['actual_kwh'].dropna())*365 # average production per year
ppy

In [ ]:
dpy/ppy

In [ ]:
union['annual_degradation_engr'].value_counts()

In [ ]:
ppy*0.005

In [ ]:
union['ages'] = (union['production_date'] - union['in_service_date']).dt.days

In [ ]:
union.head()

In [ ]:
union.groupby('contract_id').apply(lambda x: x['in_service_date']).dropna().index.get_level_values(0).unique()

In [ ]:
degradations = {}

for contract in contracts.index:
    df = union[union['contract_id'] == contract][['ages', 'actual_kwh']].dropna()
    if not df.empty:
        degradations[contract] = stats.linregress(df['ages'], df['actual_kwh'])

print(degradations)

In [ ]:
annual_degradation = {}
for elem in degradations.items():
    annual_degradation[elem[0]] = elem[1][0]/(group.mean()['actual_kwh'][elem[0]])

In [ ]:
np.mean(np.abs(list(annual_degradation.values())))

In [ ]:
len(annual_degradation.keys())

In [ ]:
union['degradation'] = union.apply(lambda row: annual_degradation[row.contract_id], axis=1)

Next, we'll create some new columns which could be useful for modeling or perhaps further analysis.

In [ ]:
union['production_year'] = union['production_date'].dt.year
pyear = pd.get_dummies(union['production_year'])
pyear.columns = ["prod" + str(name) for name in pyear]
union['production_month'] = union['production_date'].dt.month
pmonth = pd.get_dummies(union['production_month'])
pmonth.columns = ["prod" + str(name) for name in pmonth]
union['ato_year'] = union['ato_date'].dt.year
atoyear = pd.get_dummies(union['ato_year'])
atoyear.columns = ["ato" + str(name) for name in atoyear]
union['ato_month'] = union['ato_date'].dt.month
atomonth = pd.get_dummies(union['ato_month'])
atomonth.columns = ["ato" + str(name) for name in atomonth]
union = pd.concat([union, pyear, pmonth, atoyear, atomonth], axis = 1)

In [ ]:
holding_ids = pd.get_dummies(union['holding_company_id'])
holding_ids.columns = ["h" + str(name) for name in holding_ids]
project_ids = pd.get_dummies(union['project_company_id'])
project_ids.columns = ["p" + str(name) for name in project_ids]
union = pd.concat([union, holding_ids, project_ids], axis = 1)
union['holding_company_id'] = union['holding_company_id'].astype('category')
union['project_company_id'] = union['project_company_id'].astype('category')

In [ ]:
subset = union[['actual_kwh', 'expected_kwh']].copy()
sns.regplot(subset['actual_kwh'], subset['expected_kwh'])

In [ ]:
union['yield'] = union['actual_kwh']/union['size_kwdc']
union['efficiency'] = union['actual_kwh']/union['expected_kwh']

In [ ]:
# flag contracts with low production

union[union['efficiency'] < 0.1].groupby('contract_id').apply(lambda x: x['production_date'])

In [ ]:
# potential loss due to weather

union[union['expected_kwh'] < union['weather_adjusted_expected_kwh']]

# 3. Modeling

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel # for modeling
from sklearn.linear_model import Ridge, LinearRegression # also for modeling
import statsmodels.api as statsmodels

In [ ]:
correlations = union.corr()

In [ ]:
heatmap = correlations.iloc[:10, :10].dropna(how = 'all')
heatmap.dropna(axis = 1, how = 'all', inplace = True)

In [ ]:
sns.heatmap(heatmap, xticklabels=heatmap.columns, yticklabels=heatmap.columns)

In [ ]:
useful = dict()
for elem in correlations['actual_kwh'].items():
    if abs(elem[1]) > 0.1 and abs(elem[1]) != 1:
        useful[elem[0]] = elem[1]
sorted(useful.items(), key = lambda x: x[1])

In [ ]:
def mult_regression(df, column_x, column_y):
    ''' this function uses built in library functions to construct a linear 
    regression model with potentially multiple predictor variables. It outputs 
    two plots to assess the validity of the model.'''

    # If there is only one predictor variable, plot the regression line
    if len(column_x)==1:
        plt.figure()
        sns.regplot(x=column_x[0], y=column_y, data=df, marker="+",fit_reg=True,color='orange')

    # define predictors X and response Y:
    X = df[column_x]
    X = statsmodels.add_constant(X)
    Y = df[column_y]

    # construct model:
    global regressionmodel 
    regressionmodel = statsmodels.OLS(Y,X).fit() # OLS = "ordinary least squares"

    # residual plot:
    plt.figure()
    residualplot = sns.residplot(x=regressionmodel.predict(), y=regressionmodel.resid, color='green')
    residualplot.set(xlabel='Fitted values for '+column_y, ylabel='Residuals')
    residualplot.set_title('Residuals vs Fitted values',fontweight='bold',fontsize=14)

    # QQ plot:
    qqplot = statsmodels.qqplot(regressionmodel.resid,fit=True,line='45')
    qqplot.suptitle("Normal Probability (\"QQ\") Plot for Residuals",fontweight='bold',fontsize=14)

In [ ]:
mult_regression(union.dropna(), list(useful.keys()), 'actual_kwh')
regressionmodel.summary()

In [ ]:
union['production_days'] = union['production_date'].astype('int64')//1e9//60//60//24 % 365
union['ato_days'] = union['ato_date'].astype('int64')//1e9//60//60//24 % 365

In [ ]:
mult_regression(union.dropna(), ['size_kwdc', 'ato_days', 'annual_degradation_finance', 'production_days'], 'actual_kwh')
regressionmodel.summary()

In [ ]:
predictors = dict()
for elem in correlations['efficiency'].items():
    if abs(elem[1]) > 0.1 and abs(elem[1]) != 1:
        predictors[elem[0]] = elem[1]
sorted(predictors.items(), key = lambda x: x[1])
list(predictors.keys())

In [ ]:
mult_regression(union.dropna(), list(predictors.keys())[1:5], 'efficiency')
regressionmodel.summary()

In [ ]:
union.loc[union['efficiency'] > 2, 'efficiency'] = 2
union.loc[union['efficiency'] < 0, 'efficiency'] = 0

plt.hist(union['efficiency'])

In [ ]:
plt.scatter(union['production_days'], union['efficiency'])

In [ ]:
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

from kats.consts import TimeSeriesData

forecast = TimeSeriesData(union[['production_date', 'actual_kwh']].dropna(), time_col_name = 'production_date')

In [ ]:
forecast

In [ ]:
# import the param and model classes for Prophet model
from kats.models.prophet import ProphetModel, ProphetParams

# create a model param instance
params = ProphetParams(seasonality_mode='multiplicative') # additive mode gives worse results

# create a prophet model instance
m = ProphetModel(forecast, params)

# fit model simply by calling m.fit()
m.fit()

# make prediction for next 30 month
fcst = m.predict(steps=60, freq="MS")

In [ ]:
fcst.head()

In [ ]:
m.plot()

In [ ]:
forecast2 = TimeSeriesData(union[['production_date', 'efficiency']], time_col_name = 'production_date')

m2 = ProphetModel(forecast2, params)

m2.fit()

# make prediction for next 30 month
fcst2 = m2.predict(steps=60, freq="MS")

In [ ]:
fcst2.head()

In [ ]:
m2.plot()

In [ ]:
noncomms = union[union['actual_kwh'].isnull()][['contract_id', 'production_date']].groupby('contract_id')

In [ ]:
noncommdates = noncomms.apply(lambda x: x['production_date'])
noncommdates

In [ ]:
print('hello world')

In [ ]:
noncommdict = {}
for elem in contracts.index:
    try:
        if len(noncommdates[elem]) > 1:
            noncommdict[elem] = list(noncommdates[elem])
    except:
        pass
sorted(noncommdict.items())

In [ ]:
totalnoncomms = {elem: len(noncommdict[elem]) for elem in noncommdict.keys()}

In [ ]:
sorted(totalnoncomms.items(), key = lambda x: x[1])

In [ ]:
noncommlengths = {}

for elem in noncommdict.keys():
    length = 1
    for i in range(len(noncommdict[elem]) - 1):
        if noncommdict[elem][i] + pd.DateOffset(1) == noncommdict[elem][i + 1]:
            length += 1
        else:
            noncommlengths.append(length)
            length = 1
    noncommlengths.append(length)
    
print(noncommlengths)

In [ ]:
np.mean(noncommlengths)

In [ ]:
plt.hist(noncommlengths)

In [ ]:
noncommlengths.count(1), len(noncommlengths)

In [ ]:
def difference_of_means(data1, data2, tails):
    n1 = len(data1)
    n2 = len(data2)
    n = n1 + n2
    x1 = np.mean(data1)
    x2 = np.mean(data2)
    s1 = np.std(data1, ddof=1) #accounts for Bessel's correction w/ ddof=1
    s2 = np.std(data2, ddof=1)
    alpha = 0.05/(len(union['holding_company_id'].unique()) + len(union['project_company_id'].unique()))
    
    se = np.sqrt(s1**2/n1 + s2**2/n2)
    t = (x2 - x1)/se
    df = min(n1,n2) - 1 # conservative estimate from OpenIntro
    pvalue = tails*stats.t.cdf(-np.abs(t), df)

    SDpooled = np.sqrt((s1**2*(n1 - 1) + s2**2*(n2 - 1))/(n1 + n2 - 2)) # OpenIntro section 5.3.6
    Cohensd = (x2 - x1)/SDpooled
    Glassd = (x2 - x1)/s2
    Hedgesg = Cohensd*((n - 3)/(n - 2.25))*np.sqrt((n - 2)/n) #multiplies Cohen's d by correction factor to get Hedge's g

    return t, pvalue, Cohensd, Hedgesg, Glassd, pvalue < alpha

In [ ]:
print(f'holding id \t t-value \t\t p-value \t\t\t Glass\' delta \t\t significant?')
for i in union['holding_company_id'].unique():
    company = union[union['holding_company_id'] == i]['efficiency']
    others = union[union['holding_company_id'] != i]['efficiency']
    if difference_of_means(company, others, 2)[1] == 0.0:
        print(f'{i} \t\t {difference_of_means(company, others, 2)[0]} \t {difference_of_means(company, others, 2)[1]} \
    \t\t\t {difference_of_means(company, others, 2)[4]} \t {difference_of_means(company, others, 2)[-1]}')
    else:
        print(f'{i} \t\t {difference_of_means(company, others, 2)[0]} \t {difference_of_means(company, others, 2)[1]} \
    \t {difference_of_means(company, others, 2)[4]} \t {difference_of_means(company, others, 2)[-1]}')

In [ ]:
print(f'project id \t t-value \t\t p-value \t\t\t Glass\' delta \t\t significant?')
for i in union['project_company_id'].unique():
    company = union[union['project_company_id'] == i]['efficiency']
    others = union[union['project_company_id'] != i]['efficiency']
    if difference_of_means(company, others, 2)[1] == 0.0:
        print(f'{i} \t\t {difference_of_means(company, others, 2)[0]} \t {difference_of_means(company, others, 2)[1]} \
    \t\t\t {difference_of_means(company, others, 2)[4]} \t {difference_of_means(company, others, 2)[-1]}')
    else:
        print(f'{i} \t\t {difference_of_means(company, others, 2)[0]} \t {difference_of_means(company, others, 2)[1]} \
    \t {difference_of_means(company, others, 2)[4]} \t {difference_of_means(company, others, 2)[-1]}')

In [ ]:
print('alpha = ' + str(0.05/(len(union['holding_company_id'].unique()) + len(union['project_company_id'].unique()))))

In [ ]:
len(union[union['holding_company_id'] == 20]['efficiency'])

In [ ]:
np.mean(union[union['holding_company_id'] == 20]['efficiency'])

In [ ]:
np.mean(union[union['holding_company_id'] != 20]['efficiency'])

In [ ]:
np.mean(union['efficiency'])

In [ ]:
plt.hist(union[union['holding_company_id'] == 20]['efficiency'], bins = 20)

In [ ]:
plt.hist(union[union['holding_company_id'] != 20]['efficiency'], bins = 20)

In [ ]:
from statsmodels.formula.api import ols

anova = ols('efficiency ~ C(holding_company_id)', data=union).fit()
anova_table = statsmodels.stats.anova_lm(anova, typ=2)
anova_table

In [ ]:
from statsmodels.formula.api import ols

anova = ols('efficiency ~ C(project_company_id)', data=union).fit()
anova_table = statsmodels.stats.anova_lm(anova, typ=2)
anova_table

In [ ]:
type(mult_regression(union.dropna(), 'size_kwdc', 'efficiency'))
regressionmodel.summary()

In [ ]:
stats.linregress(union.dropna()['h20'], union.dropna()['efficiency'])

In [ ]:
union[union['efficiency'].isna()]

In [ ]:
temp = union.dropna()[holding_ids.columns]
temp['efficiency'] = union['efficiency']

X = temp[holding_ids.columns].to_numpy()
y = temp['efficiency'].to_numpy()

temp.head()

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X, y)

In [ ]:
regr.coef_
regr.score(X, y)

In [ ]:
testarray = np.array([0 for i in range(34)])
testarray[13] = 1
regr.predict(testarray.reshape(1, -1))

In [ ]:
ax = sns.boxplot(x='holding_company_id', y='efficiency', data=union, color='#99c2a2')
plt.show()

In [ ]:
ax = sns.boxplot(x='project_company_id', y='efficiency', data=union, color='#99c2a2')
plt.show()

In [ ]:
hax = sns.boxplot(x='holding_company_id', y='actual_kwh', data=union, color='#99c2a2')
hax = sns.swarmplot(x="holding_company_id", y="actual_kwh", data=union, color='#7d0013')
plt.show()

In [ ]:
from kats.models.sarima import SARIMAModel, SARIMAParams
warnings.simplefilter(action='ignore')

# create SARIMA param class
sparams = SARIMAParams(p = 2, d = 1, q = 1, trend = 'ct', seasonal_order = (1, 0, 1, 12))

# initiate SARIMA model
sm = SARIMAModel(data=forecast, params=sparams)

# fit SARIMA model
sm.fit()

# generate forecast values
sfcst = sm.predict(steps=30, freq="MS")

# make plot to visualize
sm.plot()